In [1]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [3]:
esp_ff = foyer.Forcefield(forcefield_files="/home/madilyn/Projects/repos/forcefields/xml_files/PIDTBT_nC16.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
molecule = mb.load("/home/madilyn/Projects/repos/forcefields/mol2files/PIDTBT_nC16_typed.mol2")

for p in molecule.particles():
    p.name = f"_{p.name}"

box = mb.fill_box(compound=molecule, n_compounds=5, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)




#setting cpu and simulation
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
#free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.0001)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_PIDTBT_nC16.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)



/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H17
  warnings.warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C9
  warnings.warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C1
  warnings.warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C26
  warnings.warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H44
  warnings.warn(
/home/madilyn/miniconda3/envs/esp_old/lib/

/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mdtraj/core/trajectory.py:438: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn('top= kwargs ignored since this file parser does not support it')
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C0 pos=([ 0.2024 -0.1112  0.1189]), 4 bonds, id: 140152330271856> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([ 0.3275 -0.0176  0.1081]), 3 bonds, id: 140152330271664> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([0.2654 0.1165 0.0631]), 3 bonds, id: 140152330260192> does n

/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C0 pos=([5.2777 3.3614 6.493 ]), 4 bonds, id: 140152329629040> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([5.1706 3.4164 6.3929]), 3 bonds, id: 140152329633744> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C2 pos=([5.2208 3.5599 6.3654]), 3 bonds, id: 140152329633648> does not have a charge.They will not be accounted for during this calculation.
  warn(
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C3 pos=([5.2786 3.6122 6.4973]), 3 bonds, id: 140152329633

/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:455: UserWarning: Parameters have been assigned to all proper dihedrals.  However, there are more parameterized dihedrals (13960) than total system dihedrals (3490).  This may be due to having multiple periodic dihedrals for a single system dihedral.
  warnings.warn(msg)
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:349: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 1420, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/home/madilyn/miniconda3/envs/esp_old/lib/python3.10/site-packages/foyer/forcefield.py:926: UserWarning: Parametrized structure has non-zero charge.Structure's total charge: 0.0023293495178222656
  warnings.warn(
*Warning*: Systems with many particle types perform poorly or r

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


In [ ]:
sim.run(1e6,write_at_start=True)